In [122]:
import pandas as pd
from services.db_service import DbService
from requests import get
from json import loads

In [123]:
personal_pool_collection = DbService().personal_pool_collection()
global_pool = DbService().pool_collection()

In [124]:
df = pd.read_csv("../raw-data/moxfield_haves.csv")
df.head()

,Count,Tradelist Count,Name,Edition,Condition,Language,Foil,Tags,Last Modified,Collector Number,Alter,Proxy,Purchase Price
0,4,4,_____ Goblin,unf,Near Mint,English,foil,NaN,2024-05-28 18:39:39.257000,107,False,False,NaN
1,4,4,Abrade,2xm,Near Mint,English,NaN,NaN,2024-05-28 18:39:39.257000,114,False,False,NaN
2,4,4,Abraded Bluffs,otj,Near Mint,English,foil,NaN,2024-05-28 18:39:22.543000,251,False,False,NaN
3,1,1,Accumulated Knowledge,a25,Near Mint,English,NaN,NaN,2024-05-28 18:38:57.807000,40,False,False,NaN
4,3,3,Accumulated Knowledge,nem,Near Mint,English,NaN,NaN,2024-06-03 03:22:09.390000,26,False,False,NaN


In [125]:
df = df[["Count", "Name"]]
df = df.rename(columns={"Count": "count", "Name": "name"})

df.head()

,count,name
0,4,_____ Goblin
1,4,Abrade
2,4,Abraded Bluffs
3,1,Accumulated Knowledge
4,3,Accumulated Knowledge


In [126]:
selected_columns = df[["count", "name"]]
cards = selected_columns.to_dict(orient="records")
cards

[{'count': 4, 'name': '_____ Goblin'},
 {'count': 4, 'name': 'Abrade'},
 {'count': 4, 'name': 'Abraded Bluffs'},
 {'count': 1, 'name': 'Accumulated Knowledge'},
 {'count': 3, 'name': 'Accumulated Knowledge'},
 {'count': 4, 'name': 'Acorn Harvest'},
 {'count': 1, 'name': 'Agony Warp'},
 {'count': 3, 'name': 'Agony Warp'},
 {'count': 4, 'name': 'All That Glitters'},
 {'count': 4, 'name': 'Alms of the Vein'},
 {'count': 4, 'name': 'Alpine Meadow'},
 {'count': 2, 'name': 'Ancestral Anger'},
 {'count': 2, 'name': 'Ancestral Anger'},
 {'count': 4, 'name': 'Ancestral Mask'},
 {'count': 4, 'name': 'Ancient Den'},
 {'count': 4, 'name': 'Ancient Grudge'},
 {'count': 4, 'name': 'Ancient Stirrings'},
 {'count': 4, 'name': 'Annoyed Altisaur'},
 {'count': 4, 'name': 'Annul'},
 {'count': 4, 'name': "Apostle's Blessing"},
 {'count': 3, 'name': 'Arashin Cleric'},
 {'count': 4, 'name': 'Arbor Elf'},
 {'count': 3, 'name': 'Arboreal Grazer'},
 {'count': 2, 'name': 'Arcane Denial'},
 {'count': 4, 'name': '

In [127]:
def get_card_data_from_scryfall(name):
    # https://api.scryfall.com/cards/search?q=%22Lightning%20Bolt%22%20is:firstprint

    scryfall_url = f"https://api.scryfall.com/cards/search?q=!\"{name}\" is:firstprint"
    cards = loads(get(scryfall_url).text)["data"]
    
    if len(cards) > 1:
        print(name)
        card = list(filter(lambda c: c["name"] == name, cards))[0]
    else:
        card = cards[0]

    released_at = card["released_at"]
    cmc = card["cmc"]
    color_identity = card["color_identity"]
    set = card["set_name"]

    type = card["type_line"]
    if "Land" in type:
        return ["L"], color_identity, cmc, released_at, set

    if "colors" in card:
        colors = card["colors"]
    else:
        colors = card["card_faces"][0]["colors"]

    return colors, color_identity, cmc, released_at, set

In [128]:
import_type = "sum"  # sum or replace


def add_or_update_card(card):
    card_from_pool = global_pool.find_one({"scryfall_name": card["name"]})
    name = card_from_pool["name"] if card_from_pool else card["name"]

    colors, color_identity, cmc, released_at, set = get_card_data_from_scryfall(
        name)

    existing_card = personal_pool_collection.find_one(
        {"scryfall_name": card["name"]})
    if not existing_card:
        personal_pool_collection.insert_one(
            {"name": name, "scryfall_name": card["name"], "count": card["count"], "colors": colors, "color_identity": color_identity, "cmc": cmc, "released_at": released_at, "set": set})
        return

    if import_type == "sum":
        card["count"] += existing_card["count"]

    personal_pool_collection.update_one({"_id": existing_card["_id"]}, {
        "$set": {"count": card["count"], "colors": colors, "color_identity": color_identity, "cmc": cmc, "released_at": released_at, "set": set}})

In [129]:
for card in cards:
    add_or_update_card(card)